In [66]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [67]:
os.environ["OPEN_AI_KEY"] = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [68]:
from langchain_community.document_loaders import TextLoader

loader=TextLoader('speech.txt')
loader


In [69]:
docs=loader.load()
docs

[Document(metadata={'source': 'speech.txt'}, page_content='Raju Gangitla is a Full Stack Developer from India, currently working at Zedblock, where he has been contributing for nearly two years. He earned his Bachelor of Technology (BTech) degree from Vignan\'s Institute of Information Technology. \ue200cite\ue202turn0search1\ue201\n\nIn his professional journey, Raju is engaged in developing AI products and is expanding his expertise in technologies such as Next.js, NestJS, MongoDB, PostgreSQL, and Google Cloud Platform (GCP). \ue200cite\ue202turn0search0\ue201\n\nRaju is also an active open-source contributor. During Hacktoberfest 2024, he participated in the oss.gg Hackathon, contributing to projects like Formbricks, Dub, Papermark, and Unkey. He documented his experiences in a blog post titled "Working by Day, Hacking by Night: My oss.gg Hackathon Experience." \ue200cite\ue202turn0search7\ue201\n\nBeyond his professional endeavors, Raju shares insights and guides on platforms like 

In [70]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
final_documents  = text_splitter.split_documents(docs)
final_documents

[Document(metadata={'source': 'speech.txt'}, page_content="Raju Gangitla is a Full Stack Developer from India, currently working at Zedblock, where he has been contributing for nearly two years. He earned his Bachelor of Technology (BTech) degree from Vignan's Institute of Information Technology. \ue200cite\ue202turn0search1\ue201\n\nIn his professional journey, Raju is engaged in developing AI products and is expanding his expertise in technologies such as Next.js, NestJS, MongoDB, PostgreSQL, and Google Cloud Platform (GCP). \ue200cite\ue202turn0search0\ue201"),
 Document(metadata={'source': 'speech.txt'}, page_content='Raju is also an active open-source contributor. During Hacktoberfest 2024, he participated in the oss.gg Hackathon, contributing to projects like Formbricks, Dub, Papermark, and Unkey. He documented his experiences in a blog post titled "Working by Day, Hacking by Night: My oss.gg Hackathon Experience." \ue200cite\ue202turn0search7\ue201'),
 Document(metadata={'source

In [71]:
import os
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("OPEN_AI_KEY"),
        api_version="2023-05-15"
        )

In [72]:
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model="gpt-4o", api_key=os.getenv("OPEN_AI_KEY"), api_version=os.getenv("OPENAI_API_VERSION"))


In [73]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(final_documents, embeddings)

vector_store

In [74]:
from langchain_core.prompts import ChatPromptTemplate

prompt  = ChatPromptTemplate.from_template(
    """
    summarise the following context in simple and short in one line 
    {context}
"""
)

In [75]:
retreiver = vector_store.as_retriever()

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [76]:
from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retreiver, document_chain)


In [80]:

result  = retriever_chain.invoke({"input":"Who is Raju Gangitla "})
result

{'input': 'Who is Raju Gangitla ',
 'context': [Document(id='98deea29-ddc8-498c-a22e-3b3cffdcd56e', metadata={'source': 'speech.txt'}, page_content="Raju Gangitla is a Full Stack Developer from India, currently working at Zedblock, where he has been contributing for nearly two years. He earned his Bachelor of Technology (BTech) degree from Vignan's Institute of Information Technology. \ue200cite\ue202turn0search1\ue201\n\nIn his professional journey, Raju is engaged in developing AI products and is expanding his expertise in technologies such as Next.js, NestJS, MongoDB, PostgreSQL, and Google Cloud Platform (GCP). \ue200cite\ue202turn0search0\ue201"),
  Document(id='779231b3-ba80-4c73-b49e-5f3be2b6b1fb', metadata={'source': 'speech.txt'}, page_content='Raju is also an active open-source contributor. During Hacktoberfest 2024, he participated in the oss.gg Hackathon, contributing to projects like Formbricks, Dub, Papermark, and Unkey. He documented his experiences in a blog post titled

In [81]:
result['answer']

'Raju Gangitla, an Indian Full Stack Developer at Zedblock, contributes to AI product development, open-source projects, and shares guides on DEV Community.'